In [4]:
alpha_name = "ResLstm_8Input_corr_filter_Shift_1_Sequence_20"
start_date = '2019-01-01'
end_date = '2021-01-01'
change = 10

In [5]:
import time
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
import numpy as np
import pandas as pd
import DataAPI
import os
input_path = r'/home/wuwenjun/factor_lib/ti0/wuwenjun/' + alpha_name
output_path = r'/home/wuwenjun/factor_lib/ti0/wuwenjun/'

alpha_list = []
trading_list = DataAPI.get_trading_days(start_date, end_date)
count = 0
for day in trading_list:
    file = os.path.join(input_path, str(day.year), day.strftime('%Y%m%d') + '.csv')
    try:
        df = pd.read_csv(file, dtype={0: str})
    except FileNotFoundError:
        count += 1
        continue
    flag = count % change
    path_output = os.path.join(output_path, str(alpha_name + "_" + str(flag)), str(day.year), day.strftime('%Y%m%d') + '.csv')
    alpha_list.append(str(alpha_name + "_" + str(flag)))
    if not os.path.exists(path_output):
        os.makedirs(os.path.dirname(path_output), exist_ok=True)
    df.to_csv(path_output, index=False, header=False)
    count += 1

In [6]:
pd.set_option('expand_frame_repr', False)
config_path = r'/home/ShareFolder/lgc/Modules/Research/config/feature_bt_template'
print('Loading the configuration from ' + config_path)
configs = namespace.load_namespace(config_path)
FT = FeatureAnalysis(configs, feature_path=r"/home/wuwenjun/factor_lib")
alpha_list = list(set(alpha_list))
print(alpha_list)
FT.load_feature_from_file(alpha_list, start_date, end_date, universe='Investable',
                                       timedelta=None, transformer=norm.standard_scale)
FT.load_return_data()

writing_path = r'/home/wuwenjun/Desktop/%s_%ddChange.txt' % (alpha_name,change)
f = open (writing_path,'w')
print("\n",file=f)
f.close()
for alpha_name in alpha_list:
    FT.get_intersection_ic(feature_name=alpha_name, truncate_fold=None, method='spearman',
                                     period=('1d', '3d', '5d'))
    ic_flag, trading_direction = FT.test_ic(alpha_name, verbose=False)
    df, df_all = FT.get_ic_summary_by_month(num=6)
    
    if trading_direction == -1:
        negative = True
    else:
        negative = False
    f = open (writing_path,'a')
    print("*"*100,end="\n"*3,file=f)
    print(alpha_name,end = "\n"*3,file=f)
    print(df,end="\n"*3,file = f)
    a = FT.get_top_return(alpha_name, negative= False, trade_type='long-only', transmission_rate=0.00025,
                            tax_rate=0.001, verbose=True,bt_price = "vwap",trade_period=6)
    print(a[1],end = "\n"*3+"*"*100 ,file = f)
    f.close()

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template


Loading Feature...:   3%|██▍                                                                                      | 132/4870 [00:00<00:06, 711.33it/s]

['ResLstm_8Input_corr_filter_Shift_1_Sequence_20_9', 'ResLstm_8Input_corr_filter_Shift_1_Sequence_20_1', 'ResLstm_8Input_corr_filter_Shift_1_Sequence_20_8', 'ResLstm_8Input_corr_filter_Shift_1_Sequence_20_5', 'ResLstm_8Input_corr_filter_Shift_1_Sequence_20_2', 'ResLstm_8Input_corr_filter_Shift_1_Sequence_20_6', 'ResLstm_8Input_corr_filter_Shift_1_Sequence_20_0', 'ResLstm_8Input_corr_filter_Shift_1_Sequence_20_7', 'ResLstm_8Input_corr_filter_Shift_1_Sequence_20_4', 'ResLstm_8Input_corr_filter_Shift_1_Sequence_20_3']


Loading Feature...:  61%|█████████████████████████████████████████████████████▋                                  | 2970/4870 [00:02<00:02, 904.78it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Calculating IC value...: 100%|█████████████████████████████████████████████████████████████████████████████████████| 487/487 [00:00<00:00, 640.97it/s]



IC table for feature "ResLstm_8Input_corr_filter_Shift_1_Sequence_20_9" (mean std IR):
+-------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|      period       |  IC_ResLstm_1d  |  IC_ResLstm_2d  |  IC_ResLstm_3d  |  IC_ResLstm_4d  |  IC_ResLstm_5d  |
+===================+=================+=================+=================+=================+=================+
| 20190115-20190630 | 3.92 6.81 0.57  | 5.27 7.87 0.67  | 5.45 7.55 0.72  | 6.80 8.23 0.83  | 6.65 7.80 0.85  |
+-------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
| 20190701-20191231 | 2.91 3.79 0.77  | 5.23 2.83 1.85  | 4.87 3.08 1.58  | 5.11 3.69 1.38  | 5.29 3.46 1.53  |
+-------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
| 20200101-20200630 | 1.09 5.02 0.22  | 2.25 5.44 0.41  | 2.65 5.43 0.49  | 4.05 5.59 0.72  | 5.06 5.69 0.89  |
+---------------

...


ResLstm_8Input_corr_filter_Shift_1_Sequence_20_4 back-test result for long-only mode:

Back test summary: "long-only" mode
+-------------------+-------------+----------+-----------+------------+----------+-------------+-------------------------+-------------------+------------+
|      period       |  stock_num  |  return  |  ret_std  |  ret_year  |  sharpe  |  win_ratio  |      max_dd|period      |  max_dd_day|date  |  turnover  |
+===================+=============+==========+===========+============+==========+=============+=========================+===================+============+
| 20190102-20190630 |     210     |   8.81   |   5.75    |   18.81    |   3.23   |    54.70    | 3.09(20190109-20190131) |  1.03(20190131)   |   14.08    |
+-------------------+-------------+----------+-----------+------------+----------+-------------+-------------------------+-------------------+------------+
| 20190701-20191231 |     213     |   1.60   |   3.77    |    3.07    |   0.85   |    59.20    |

Calculating IC value...: 100%|█████████████████████████████████████████████████████████████████████████████████████| 487/487 [00:00<00:00, 756.90it/s]



IC table for feature "ResLstm_8Input_corr_filter_Shift_1_Sequence_20_3" (mean std IR):
+-------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|      period       |  IC_ResLstm_1d  |  IC_ResLstm_2d  |  IC_ResLstm_3d  |  IC_ResLstm_4d  |  IC_ResLstm_5d  |
+===================+=================+=================+=================+=================+=================+
| 20190107-20190630 | 4.53 5.04 0.90  | 4.57 4.89 0.93  | 6.39 6.43 0.99  | 6.38 6.85 0.93  | 7.28 6.96 1.05  |
+-------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
| 20190701-20191231 | 1.72 4.46 0.39  | 2.28 3.90 0.58  | 2.50 4.17 0.60  | 2.78 3.90 0.71  | 3.79 3.97 0.96  |
+-------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
| 20200101-20200630 | 3.38 6.21 0.54  | 3.47 5.87 0.59  | 3.98 6.65 0.60  | 4.51 6.86 0.66  | 5.67 6.74 0.84  |
+---------------

Getting BT result for date: 20201231...: 100%|██████████████████████████████████████████████████████████████████████| 487/487 [00:05<00:00, 94.60it/s]



ResLstm_8Input_corr_filter_Shift_1_Sequence_20_3 back-test result for long-only mode:

Back test summary: "long-only" mode
+-------------------+-------------+----------+-----------+------------+----------+-------------+-------------------------+-------------------+------------+
|      period       |  stock_num  |  return  |  ret_std  |  ret_year  |  sharpe  |  win_ratio  |      max_dd|period      |  max_dd_day|date  |  turnover  |
+===================+=============+==========+===========+============+==========+=============+=========================+===================+============+
| 20190102-20190630 |     208     |   7.92   |   6.00    |   16.83    |   2.77   |    59.83    | 3.25(20190111-20190131) |  1.16(20190131)   |   14.31    |
+-------------------+-------------+----------+-----------+------------+----------+-------------+-------------------------+-------------------+------------+
| 20190701-20191231 |     211     |  -1.13   |   3.94    |   -2.15    |  -0.58   |    55.20    |